#*Exercise:3*

#*Regularized OLSLR*:
\begin{equation}
min_x f_\lambda(x) = \frac{\lambda}{2}x^\top x + \frac{1}{2} \|Ax - y\|_2^2
\end{equation}
Let  $x^*_{f_\lambda} =argmin_x f_\lambda(x)$

##***Theory***: 
#*Regularized OLSLR*
>* **Required:** $\min_{x}f_\lambda(x)= \frac{\lambda}{2}x^\top x + \frac{1}{2}\|Ax -y\|_2^2$
>* **Gradient of the above function:** $\nabla f_\lambda(x) = \lambda x + <A, Ax-y> = \lambda x + A^T Ax - A^\top y$
>* **Hessian of the function:** $\nabla^2 f_\lambda(x) = \lambda I + A^T A$


#*Question 1:* 
It is given that, the regularized problem can be written as 
\begin{equation}
min_x f_\lambda(x) = min_x \sum_{i=1}^{N}f_i(x)
\end{equation}
An appropriate choice of $f_i(x)$ is given as follows:
\begin{equation}
f_i(x) = \frac{\lambda}{2N}\sum_{j=1}^{d}x_j^2 + \frac{1}{2} \big[(\sum_{j = 1}^{d} A_{ij}x_j) - y_i \big]^2
\end{equation}

#*Question 2:*
The $k^{th}$ component of the gradient of $f_i(x)$ is as follows:
\begin{equation}
(g_i(x))_k = (\nabla_x f_i(x))_k = \frac{\lambda}{N}x_k + \big[(\sum_{j = 1}^{d} A_{ij}x_j) - y_i \big]A_{ik}
\end{equation}

#*Importing required packages*

In [25]:
import numpy as np 
import timeit
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import pandas as pd
import seaborn as sns
from tqdm import tqdm
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings("ignore")

In [26]:
def eval_f_i(lambd, x, y_i, data_point, N, d):
    return 0.5*(np.subtract(np.dot(data_point, x), y_i)**2) + (lambd/2*N)*np.dot(x.T, x)


In [27]:
def eval_g_i(lambd, x, y_i, data_point, N, d):
    return np.array([np.subtract(np.dot(data_point, x), y_i)*data_point[k] + (lambd/N)*x[k] for k in range(d)])

#*Question 3:*

In [28]:
def alg_labs(lambda_reg, epochs, modified = False):
    np.random.seed(1000) #for repeatability
    N = 200
    d = 20000 #Consider the dimension which caused failure in the previous experiment
    eps = np.random.randn(N,1) #random noise

    #Create data matrix, label vector
    A = np.random.randn(N,d)
    #Normalize the columns
    for j in range(A.shape[1]):
      A[:,j] = A[:,j]/np.linalg.norm(A[:,j])

    xorig = np.ones( (d,1) )
    y = np.dot(A,xorig) + eps

    #initialize the optimization variable to be used in the new algo ALG-LAB7
    x = np.zeros((d,1))
    t = 1
    arr = np.arange(N) #index array

    start = timeit.default_timer() #start the timer
    for epoch in range(epochs):
        np.random.shuffle(arr) #shuffle every epoch
        for i in np.nditer(arr): #Pass through the data points
            data_point = A[i, :]
            y_i = y[i]
            g_i = eval_g_i(lambda_reg, x, y_i, data_point, N, d)
            #print(epoch)
            if modified == True:
                x = x - np.multiply((1/(N*t)), g_i)
                t = t+1
                if t>10**4:
                  t = 1
            else:
                x = x - np.multiply((1/t), g_i)
                t = t+1
                if t>10**4:
                  t = 1
            x_alglab7 = x
            g_opt = eval_g_i(lambda_reg, x, y_i, data_point, N, d)
            y_diff = (np.linalg.norm(np.subtract(np.dot(A,x_alglab7), y)))**2
            x_diff = (np.linalg.norm(x_alglab7 - xorig))**2
            print(f"||∇fλ(x)||:", g_opt)
            print(f"||Ax* - y||^2: {y_diff}")
            print("L2 norm difference ||x* - x_orig||^2:", x_diff)
    print(f"\nDone for total epochs: {epochs}\n")
    # print(f"Function value at epoch {epoch+1}: ", np.sum([eval_f_i(lambda_reg, x, y[i], A[i, :], N, d) for i in range(N)]))

    alglab7time = timeit.default_timer() - start #time is in seconds
    x_alglab7 = x
    g_opt = eval_g_i(lambda_reg, x, y_i, data_point, N, d)
    y_diff = (np.linalg.norm(np.subtract(np.dot(A,x_alglab7), y)))**2
    x_diff = (np.linalg.norm(x_alglab7 - xorig))**2
    return alglab7time,g_opt, y_diff, x_diff

In [29]:
lambda_reg = 0.001
epochs = 10**4
time, g_opt, y_diff, x_diff = alg_labs(lambda_reg, epochs)
print("For the given version with 1/t:\n")
print(f"Time taken to solve: {time}")
print(f"||∇fλ(x)||:", g_opt)
print(f"||Ax* - y||^2: {y_diff}")
print("L2 norm difference ||x* - x_orig||^2:", x_diff)
print('')

Streaming output truncated to the last 5000 lines.
 [-7.33103185]
 [ 2.39413586]
 [-2.00258992]]
||Ax* - y||^2: 133951.0929123214
L2 norm difference ||x* - x_orig||^2: 21257.041837325123
||∇fλ(x)||: [[ 0.05459798]
 [-0.41173593]
 [ 0.14520253]
 ...
 [-0.35048144]
 [ 0.17668251]
 [-0.09350927]]
||Ax* - y||^2: 133950.81720850646
L2 norm difference ||x* - x_orig||^2: 21257.027044611437
||∇fλ(x)||: [[-0.22551825]
 [-0.15387796]
 [-0.00233888]
 ...
 [-0.03074708]
 [-0.01149797]
 [-0.01615401]]
||Ax* - y||^2: 133950.56843048648
L2 norm difference ||x* - x_orig||^2: 21257.02311006448
||∇fλ(x)||: [[ 0.47617741]
 [-0.0771662 ]
 [ 0.09723071]
 ...
 [-0.08942804]
 [ 0.21377933]
 [ 0.29671854]]
||Ax* - y||^2: 133950.50737461535
L2 norm difference ||x* - x_orig||^2: 21257.004813184394
||∇fλ(x)||: [[ 0.00142537]
 [-0.07695248]
 [-0.01927535]
 ...
 [ 0.03084349]
 [ 0.06747857]
 [ 0.04470567]]
||Ax* - y||^2: 133950.62409790637
L2 norm difference ||x* - x_orig||^2: 21257.002067611058
||∇fλ(x)||: [[-0.2

KeyboardInterrupt: ignored

In [30]:
time_modif, g_modif, y_diff_modif, x_diff_modif = alg_labs(lambda_reg, epochs, modified = True)
print("For the modified version with 1/Nt:\n")
print(f"Time taken to solve: {time_modif}")
print(f"||∇fλ(x)||:", g_modif)
print(f"||Ax* - y||^2: {y_diff_modif}")
print("L2 norm difference ||x* - x_orig||^2:", x_diff_modif)
print('')

Streaming output truncated to the last 5000 lines.
 [-2.23986386]
 [-1.94650535]
 [-1.58712831]]
||Ax* - y||^2: 14964.318398831265
L2 norm difference ||x* - x_orig||^2: 19993.10353895188
||∇fλ(x)||: [[-0.01200531]
 [ 0.00228196]
 [ 0.01488073]
 ...
 [ 0.02230537]
 [-0.00746033]
 [-0.00397515]]
||Ax* - y||^2: 14964.318467411042
L2 norm difference ||x* - x_orig||^2: 19993.103538165178
||∇fλ(x)||: [[ 0.00308525]
 [-0.1815159 ]
 [-0.18630142]
 ...
 [ 0.02937756]
 [ 0.09196559]
 [-0.05133833]]
||Ax* - y||^2: 14964.315257720034
L2 norm difference ||x* - x_orig||^2: 19993.103532758905
||∇fλ(x)||: [[ 0.37159812]
 [-0.14409876]
 [ 0.45405044]
 ...
 [-0.03326935]
 [-0.90670481]
 [-0.22995608]]
||Ax* - y||^2: 14964.289246706869
L2 norm difference ||x* - x_orig||^2: 19993.103301291114
||∇fλ(x)||: [[ 0.31589262]
 [-0.11789022]
 [-0.05081289]
 ...
 [ 0.03817766]
 [-0.03247365]
 [ 0.32768383]]
||Ax* - y||^2: 14964.27887116911
L2 norm difference ||x* - x_orig||^2: 19993.10318439439
||∇fλ(x)||: [[-0.33

KeyboardInterrupt: ignored

#*Question 4:*
The second one is displayed with a proper indication that it is for 1/Nt

In [31]:
epoch_list = [10**6, 10**8, 10**10]
lambda_reg = 0.001
time = [0 for _ in range(len(epoch_list))]
y_diff = [0 for _ in range(len(epoch_list))]
x_diff = [0 for _ in range(len(epoch_list))]

for epoch in range(len(epoch_list)):
    time[epoch], y_diff[epoch], x_diff[epoch] = alg_labs(lambda_reg, epoch_list[epoch])

df = pd.DataFrame(list(zip(epoch_list, time, y_diff, x_diff)), columns = ['Epochs','Computation time','||∇fλ(x)||', '||Ax-y||^2', '||x* - x_orig||^2'])
df.set_index(np.arange(1,len(epoch_list)+1))

Streaming output truncated to the last 5000 lines.
 [ 0.04919513]
 [-0.35548727]
 [-0.03666726]]
||Ax* - y||^2: 191528.8836247774
L2 norm difference ||x* - x_orig||^2: 21857.942312138646
||∇fλ(x)||: [[-0.02698081]
 [-0.00653931]
 [-0.11520589]
 ...
 [ 0.00892943]
 [ 0.05884122]
 [-0.02755583]]
||Ax* - y||^2: 191528.93603047368
L2 norm difference ||x* - x_orig||^2: 21857.942641267815
||∇fλ(x)||: [[-0.18317293]
 [-0.2095769 ]
 [-0.12219262]
 ...
 [ 0.44133321]
 [ 0.37810334]
 [ 0.37525523]]
||Ax* - y||^2: 191527.4127731687
L2 norm difference ||x* - x_orig||^2: 21857.833798733867
||∇fλ(x)||: [[-1.58825341]
 [ 0.59038086]
 [ 1.59252216]
 ...
 [-1.07830519]
 [-0.72875014]
 [-0.29311043]]
||Ax* - y||^2: 191482.23052271045
L2 norm difference ||x* - x_orig||^2: 21857.31530160492
||∇fλ(x)||: [[ 0.02241825]
 [-0.04470137]
 [ 0.00051149]
 ...
 [-0.05286975]
 [-0.06247432]
 [-0.04715935]]
||Ax* - y||^2: 191482.22091498776
L2 norm difference ||x* - x_orig||^2: 21857.31554662554
||∇fλ(x)||: [[-0.013

KeyboardInterrupt: ignored

In [32]:
epoch_list = [10**6, 10**8, 10**10]
lambda_reg = 0.001
time_modif = [0 for _ in range(len(epoch_list))]
y_diff_modif = [0 for _ in range(len(epoch_list))]
x_diff_modif = [0 for _ in range(len(epoch_list))]

for epoch in range(len(epoch_list)):
    time_modif[epoch], y_diff_modif[epoch], x_diff_modif[epoch] = alg_labs(lambda_reg, epoch_list[epoch], modified = True)
print("\nFor the modified version with 1/Nt:\n")
df_modif = pd.DataFrame(list(zip(epoch_list, time_modif, y_diff_modif, x_diff_modif)), columns = ['Epochs','Computation time','||∇fλ(x)||', '||Ax-y||^2', '||x* - x_orig||^2'])
df_modif.set_index(np.arange(1,len(epoch_list)+1))

Streaming output truncated to the last 5000 lines.
 [ 0.16211477]
 [ 0.07239801]
 [-0.17813942]]
||Ax* - y||^2: 15085.122548036328
L2 norm difference ||x* - x_orig||^2: 19994.27656638504
||∇fλ(x)||: [[-0.4157413 ]
 [-0.81200861]
 [-0.64401467]
 ...
 [-0.86927273]
 [-0.84471374]
 [-0.47105043]]
||Ax* - y||^2: 15084.827643098804
L2 norm difference ||x* - x_orig||^2: 19994.27373309204
||∇fλ(x)||: [[-2.17092361]
 [-0.58799856]
 [-0.93726405]
 ...
 [ 0.65763129]
 [ 1.26294988]
 [ 2.23610269]]
||Ax* - y||^2: 15083.628529567677
L2 norm difference ||x* - x_orig||^2: 19994.262341668906
||∇fλ(x)||: [[ 0.68496436]
 [ 0.04074554]
 [-0.51195963]
 ...
 [ 0.09953317]
 [-0.5930597 ]
 [ 0.74758991]]
||Ax* - y||^2: 15083.526108331023
L2 norm difference ||x* - x_orig||^2: 19994.261483809092
||∇fλ(x)||: [[-0.16878074]
 [-0.32442937]
 [-0.39013927]
 ...
 [-1.21764752]
 [ 0.39765406]
 [-0.3326201 ]]
||Ax* - y||^2: 15083.382597752461
L2 norm difference ||x* - x_orig||^2: 19994.260256825266
||∇fλ(x)||: [[-0.0

KeyboardInterrupt: ignored

#*Question 5:* 
The second one is displayed with a proper indication that it is for 1/Nt

In [33]:
epochs = 10**9
lambda_list = [1000, 100, 10, 1, 0.1, 0.01, 0.001]

time_2 = [0 for _ in range(len(lambda_list))]
y_diff_2 = [0 for _ in range(len(lambda_list))]
x_diff_2 = [0 for _ in range(len(lambda_list))]

for lambda_reg in range(len(lambda_list)):
    time_2[lambda_reg], y_diff_2[lambda_reg], x_diff_2[lambda_reg] = alg_labs(lambda_reg, epochs)

df2 = pd.DataFrame(list(zip(lambda_list, time_2, y_diff_2, x_diff_2)), columns = ['Lambda value','Computation time', '||Ax-y||^2', '||x* - x_orig||^2'])
df2.set_index(np.arange(1,len(lambda_list)+1))


Streaming output truncated to the last 5000 lines.
 [ 0.23223963]
 [ 0.0833043 ]
 [ 0.57160319]]
||Ax* - y||^2: 1923291.9446541553
L2 norm difference ||x* - x_orig||^2: 39605.09384531693
||∇fλ(x)||: [[-0.59491712]
 [ 0.09641048]
 [-0.12147581]
 ...
 [ 0.11172664]
 [-0.26708935]
 [-0.37071042]]
||Ax* - y||^2: 1923036.9293856143
L2 norm difference ||x* - x_orig||^2: 39604.29846802587
||∇fλ(x)||: [[-0.12489647]
 [-0.41042245]
 [-0.22822859]
 ...
 [-0.36643721]
 [-0.68090592]
 [-0.22618609]]
||Ax* - y||^2: 1922853.2729884507
L2 norm difference ||x* - x_orig||^2: 39603.63253965431
||∇fλ(x)||: [[ 0.07991792]
 [ 0.00779879]
 [-0.03122568]
 ...
 [-0.15292808]
 [ 0.23174599]
 [ 0.00459389]]
||Ax* - y||^2: 1922804.206031603
L2 norm difference ||x* - x_orig||^2: 39603.54980210135
||∇fλ(x)||: [[-0.41196989]
 [ 0.6601748 ]
 [ 0.01896597]
 ...
 [ 0.10305651]
 [ 0.56896455]
 [-0.00839757]]
||Ax* - y||^2: 1922727.5455419116
L2 norm difference ||x* - x_orig||^2: 39602.85049288256
||∇fλ(x)||: [[-2.51412

KeyboardInterrupt: ignored

In [34]:
time_2_modif = [0 for _ in range(len(lambda_list))]
y_diff_2_modif = [0 for _ in range(len(lambda_list))]
x_diff_2_modif = [0 for _ in range(len(lambda_list))]

for lambda_reg in range(len(lambda_list)):
    time_2_modif[lambda_reg], y_diff_2_modif[lambda_reg], x_diff_2_modif[lambda_reg] = alg_labs(lambda_reg, epochs, modified = True)
print("\nFor the modified version with 1/Nt:\n")
df2_modif = pd.DataFrame(list(zip(lambda_list, time_2_modif, y_diff_2_modif, x_diff_2_modif)), columns = ['Lambda value','Computation time', '||Ax-y||^2', '||x* - x_orig||^2'])
df2_modif.set_index(np.arange(1,len(lambda_list)+1))

Streaming output truncated to the last 5000 lines.
 [ 0.69129617]
 [ 0.49634671]
 [ 0.02240564]]
||Ax* - y||^2: 15011.006172779806
L2 norm difference ||x* - x_orig||^2: 19993.55675012297
||∇fλ(x)||: [[ 0.14622833]
 [ 1.08893984]
 [-0.63881083]
 ...
 [-0.00326615]
 [-0.02356766]
 [ 0.82583033]]
||Ax* - y||^2: 15010.936947441967
L2 norm difference ||x* - x_orig||^2: 19993.556005416267
||∇fλ(x)||: [[-0.19958957]
 [-0.32792821]
 [-0.0431729 ]
 ...
 [ 0.22365678]
 [-0.12248278]
 [-0.16764875]]
||Ax* - y||^2: 15010.886604562704
L2 norm difference ||x* - x_orig||^2: 19993.5555149269
||∇fλ(x)||: [[-0.69710288]
 [-0.05377041]
 [-0.20982309]
 ...
 [-1.05829154]
 [-0.52990184]
 [ 0.50239209]]
||Ax* - y||^2: 15010.823623369417
L2 norm difference ||x* - x_orig||^2: 19993.554923463835
||∇fλ(x)||: [[-0.53758382]
 [ 0.74667905]
 [-1.79885326]
 ...
 [-1.30660627]
 [ 0.97307817]
 [-0.7699102 ]]
||Ax* - y||^2: 15010.618488861694
L2 norm difference ||x* - x_orig||^2: 19993.5532663904
||∇fλ(x)||: [[-0.0196

KeyboardInterrupt: ignored

#*Question 6:*

>* The Newton and BFGS algorithm took a lot of time for the failure dimension $d =20000$. We observe that the algorithm is computing but slowly. This resolved the ram crashing issue but it is very slow, that, for the first time I ran it for 7hrs. Then I thought to print everything so that it would be evident that the algorithm is running and converging. We can see that the squared norm of the difference between finally obtained optimal point and original $x$ is very high, and so is the value of  $\|Ax^* - y\|^2_2$.
>* The initialization of  $x$ is also an important factor that affects the squared norm differences. 

#*Question 7:*
>* For the given problem,  $f_{\lambda}=\sum_{i=1}^Nf_i(x)$ i.e. it can be written as sum of the functional values over all the available data points . After every epoch, we iterate over all the data points, and update x using the rule $x=x-\frac{1}{t}\nabla f_i(x)$. Considering t as a steplength, we can assume that after every epoch, the update is equivalent to the Gradient Descent Algorithm.
>* The complexity of the algorithm is lessone encountered in the previous exercise because data points are handled one by one, instead of being handled all at once.